In [ ]:
# ------------------------------------------------------
# Neo4j 그래프 데이터베이스 생성 시스템
# ------------------------------------------------------

# 필요한 라이브러리 설치 (처음 실행시)
# !pip install neo4j pandas xmltodict

import xml.etree.ElementTree as ET
import pandas as pd
import json
from neo4j import GraphDatabase
from pathlib import Path
import logging
from datetime import datetime
import xmltodict

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 파일 경로
XML_FILE_PATH = Path("./data/employee_relationships.xml")

# Neo4j 연결 설정
NEO4J_CONFIG = {
    "uri": "bolt://18.208.202.177:7687",            # Neo4j 기본 포트
    "username": "neo4j",                            # 기본 사용자명
    "password": "handwriting-collar-dive"           # 설치시 설정한 비밀번호로 변경 필요
}

class Neo4jManager:
    """Neo4j 데이터베이스 연결 및 관리 클래스 (연결 복구 기능 추가)"""
    
    def __init__(self, uri, username, password):
        self.uri = uri
        self.username = username
        self.password = password
        self.driver = None
        self._connect()
    
    def _connect(self):
        """Neo4j 연결 설정"""
        try:
            # 연결 풀 설정 추가 (안정성 향상)
            self.driver = GraphDatabase.driver(
                self.uri, 
                auth=(self.username, self.password),
                max_connection_lifetime=3600,  # 1시간
                max_connection_pool_size=50,   # 연결 풀 크기
                connection_acquisition_timeout=60  # 연결 획득 타임아웃
            )
            logger.info("Neo4j 연결 성공")
            
            # 연결 테스트
            with self.driver.session() as session:
                result = session.run("RETURN 1")
                logger.info("Neo4j 서버 응답 확인")
                
        except Exception as e:
            logger.error(f"Neo4j 연결 실패: {str(e)}")
            raise
    
    def verify_connection(self):
        """연결 상태 확인 및 복구"""
        try:
            self.driver.verify_connectivity()
            return True
        except Exception as e:
            logger.warning(f"연결 확인 실패: {str(e)}")
            try:
                logger.info("연결 재설정 시도...")
                if self.driver:
                    self.driver.close()
                self._connect()
                return True
            except Exception as reconnect_error:
                logger.error(f"연결 재설정 실패: {str(reconnect_error)}")
                return False
    
    def close(self):
        if self.driver:
            self.driver.close()
            logger.info("Neo4j 연결 종료")
    
    def clear_database(self):
        """데이터베이스 초기화 (기존 데이터 삭제)"""
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
            logger.info("데이터베이스 초기화 완료")
    
    def create_constraints_and_indexes(self):
        """제약조건 및 인덱스 생성"""
        constraints_queries = [
            "CREATE CONSTRAINT IF NOT EXISTS FOR (e:Employee) REQUIRE e.employee_id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (d:Department) REQUIRE d.name IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Project) REQUIRE p.project_id IS UNIQUE",
            "CREATE INDEX IF NOT EXISTS FOR (e:Employee) ON (e.risk_tier)",
            "CREATE INDEX IF NOT EXISTS FOR (e:Employee) ON (e.job_level)",
            "CREATE INDEX IF NOT EXISTS FOR (e:Employee) ON (e.persona)"
        ]
        
        with self.driver.session() as session:
            for query in constraints_queries:
                try:
                    session.run(query)
                    logger.info(f"실행 완료: {query}")
                except Exception as e:
                    logger.warning(f"제약조건/인덱스 생성 실패: {str(e)}")

# Neo4j 매니저 초기화 (실제 환경에 맞게 설정 변경 필요)
print("Neo4j 연결 설정:")
print("1. Neo4j Desktop 또는 Neo4j Community Server가 실행 중이어야 합니다")
print("2. 아래 설정을 실제 환경에 맞게 수정하세요:")
print(f"   - URI: {NEO4J_CONFIG['uri']}")
print(f"   - Username: {NEO4J_CONFIG['username']}")
print(f"   - Password: {NEO4J_CONFIG['password']}")
print("\n주의: 비밀번호를 실제 Neo4j 비밀번호로 변경하세요!")

Neo4j 연결 설정:
1. Neo4j Desktop 또는 Neo4j Community Server가 실행 중이어야 합니다
2. 아래 설정을 실제 환경에 맞게 수정하세요:
   - URI: bolt://18.208.202.177:7687
   - Username: neo4j
   - Password: handwriting-collar-dive

주의: 비밀번호를 실제 Neo4j 비밀번호로 변경하세요!


In [2]:
# ------------------------------------------------------
# XML 파일 파싱 및 데이터 구조 분석
# ------------------------------------------------------

def parse_xml_to_graph_data(xml_file_path):
    """XML 파일을 파싱하여 그래프 생성용 데이터 추출"""
    
    if not xml_file_path.exists():
        raise FileNotFoundError(f"XML 파일을 찾을 수 없습니다: {xml_file_path}")
    
    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        
        # 데이터 저장 구조
        graph_data = {
            'employees': [],
            'departments': set(),
            'projects': [],
            'hierarchy_relationships': [],
            'collaboration_relationships': [],
            'project_relationships': []
        }
        
        # 프로젝트 레지스트리 추출
        projects_section = root.find('ProjectRegistry')
        if projects_section is not None:
            for project_elem in projects_section.findall('Project'):
                project_data = {
                    'project_id': project_elem.get('project_id'),
                    'project_name': project_elem.get('project_name', project_elem.get('project_id')),
                    'project_type': project_elem.get('project_type'),
                    'team_size': int(project_elem.get('team_size', 0)),
                    'start_date': project_elem.get('start_date'),
                    'end_date': project_elem.get('end_date'),
                    'status': project_elem.get('status'),
                    'priority': project_elem.get('priority', 'medium')
                }
                graph_data['projects'].append(project_data)
        
        # 직원 정보 및 관계 추출
        employees_section = root.find('Employees')
        if employees_section is not None:
            for emp_elem in employees_section.findall('Employee'):
                
                # 직원 기본 정보
                employee_data = {
                    'employee_id': emp_elem.get('id'),
                    'name': emp_elem.get('name'),
                    'department': emp_elem.get('department'),
                    'job_role': emp_elem.get('job_role'),
                    'job_level': int(emp_elem.get('job_level', 1)),
                    'years_at_company': int(emp_elem.get('years_at_company', 0)),
                    'risk_tier': emp_elem.get('risk_tier'),
                    'persona': emp_elem.get('persona')
                }
                graph_data['employees'].append(employee_data)
                graph_data['departments'].add(emp_elem.get('department'))
                
                # 계층 관계 추출
                hierarchy = emp_elem.find('HierarchyRelationships')
                if hierarchy is not None:
                    # 상사 관계
                    reports_to = hierarchy.find('ReportsTo')
                    if reports_to is not None:
                        hierarchy_rel = {
                            'subordinate_id': emp_elem.get('id'),
                            'manager_id': reports_to.get('manager_id'),
                            'reporting_frequency': reports_to.get('reporting_frequency'),
                            'relationship_start': reports_to.get('reporting_since', '2023-01-01')
                        }
                        graph_data['hierarchy_relationships'].append(hierarchy_rel)
                
                # 협업 관계 추출
                collab = emp_elem.find('CollaborationRelationships')
                if collab is not None:
                    for colleague_elem in collab.findall('Colleague'):
                        collab_rel = {
                            'employee1_id': emp_elem.get('id'),
                            'employee2_id': colleague_elem.get('colleague_id'),
                            'collaboration_strength': float(colleague_elem.get('collaboration_strength', 0)),
                            'interaction_frequency': float(colleague_elem.get('interaction_frequency', 0)),
                            'collaboration_type': colleague_elem.get('collaboration_type'),
                            'relationship_quality': colleague_elem.get('relationship_quality'),
                            'common_projects': int(colleague_elem.get('common_projects', 0))
                        }
                        graph_data['collaboration_relationships'].append(collab_rel)
                
                # 프로젝트 참여 관계 추출
                projects = emp_elem.find('ProjectRelationships')
                if projects is not None:
                    for project_elem in projects.findall('ProjectParticipation'):
                        project_rel = {
                            'employee_id': emp_elem.get('id'),
                            'project_id': project_elem.get('project_id'),
                            'role_in_project': project_elem.get('role_in_project'),
                            'contribution_level': float(project_elem.get('contribution_level', 0)),
                            'project_start': project_elem.get('project_start'),
                            'project_end': project_elem.get('project_end'),
                            'project_status': project_elem.get('project_status')
                        }
                        graph_data['project_relationships'].append(project_rel)
        
        # 부서 정보를 리스트로 변환
        graph_data['departments'] = [{'name': dept} for dept in graph_data['departments']]
        
        logger.info(f"XML 파싱 완료:")
        logger.info(f"  - 직원: {len(graph_data['employees'])}명")
        logger.info(f"  - 부서: {len(graph_data['departments'])}개")
        logger.info(f"  - 프로젝트: {len(graph_data['projects'])}개")
        logger.info(f"  - 계층 관계: {len(graph_data['hierarchy_relationships'])}개")
        logger.info(f"  - 협업 관계: {len(graph_data['collaboration_relationships'])}개")
        logger.info(f"  - 프로젝트 참여: {len(graph_data['project_relationships'])}개")
        
        return graph_data
    
    except Exception as e:
        logger.error(f"XML 파싱 실패: {str(e)}")
        raise

# XML 데이터 파싱 실행
graph_data = parse_xml_to_graph_data(XML_FILE_PATH)

# 샘플 데이터 미리보기
print("\n=== 파싱된 데이터 샘플 ===")
print(f"직원 샘플 (첫 3명):")
for i, emp in enumerate(graph_data['employees'][:3]):
    print(f"  {i+1}. {emp['name']} - {emp['department']} - {emp['job_role']} (Level {emp['job_level']})")

print(f"\n부서 목록:")
for dept in graph_data['departments']:
    print(f"  - {dept['name']}")

print(f"\n프로젝트 샘플 (첫 3개):")
for i, proj in enumerate(graph_data['projects'][:3]):
    print(f"  {i+1}. {proj['project_id']} - {proj['project_type']} - 팀규모: {proj['team_size']}")

INFO:__main__:XML 파싱 완료:
INFO:__main__:  - 직원: 1470명
INFO:__main__:  - 부서: 3개
INFO:__main__:  - 프로젝트: 25개
INFO:__main__:  - 계층 관계: 295개
INFO:__main__:  - 협업 관계: 1098084개
INFO:__main__:  - 프로젝트 참여: 138개



=== 파싱된 데이터 샘플 ===
직원 샘플 (첫 3명):
  1. Employee_1 - Sales - Sales Executive (Level 2)
  2. Employee_2 - Research & Development - Research Scientist (Level 2)
  3. Employee_4 - Research & Development - Laboratory Technician (Level 1)

부서 목록:
  - Research & Development
  - Human Resources
  - Sales

프로젝트 샘플 (첫 3개):
  1. PRJ_001 - single_department - 팀규모: 8
  2. PRJ_002 - single_department - 팀규모: 5
  3. PRJ_003 - cross_department - 팀규모: 8


In [ ]:
# ------------------------------------------------------
# Neo4j 연결 및 그래프 생성 (수정된 버전)
# ------------------------------------------------------

# 전역 변수로 neo4j_manager 선언
neo4j_manager = None

def create_neo4j_graph_fixed(graph_data, neo4j_config):
    """Neo4j 그래프 생성 (수정된 버전)"""
    
    global neo4j_manager  # 전역 변수로 설정
    
    try:
        print("Neo4j 연결 시도 중...")
        
        # Neo4j 연결
        neo4j_manager = Neo4jManager(
            neo4j_config['uri'], 
            neo4j_config['username'], 
            neo4j_config['password']
        )
        
        print("연결 성공! 그래프 생성 시작...")
        
        # 1. 데이터베이스 초기화
        print("\n1. 기존 데이터 삭제 중...")
        neo4j_manager.clear_database()
        
        # 2. 제약조건 및 인덱스 생성
        print("2. 제약조건 및 인덱스 생성 중...")
        neo4j_manager.create_constraints_and_indexes()
        
        # 함수들이 정의되어 있는지 확인
        functions_to_check = [
            'create_departments_nodes', 
            'create_employees_nodes', 
            'create_projects_nodes',
            'create_employee_department_relationships'
        ]
        
        missing_functions = []
        for func_name in functions_to_check:
            if func_name not in globals():
                missing_functions.append(func_name)
        
        if missing_functions:
            print(f"❌ 다음 함수들이 정의되지 않았습니다: {missing_functions}")
            print("셀 3 (노드 생성 함수들)을 먼저 실행하세요!")
            return None
        
        # 3. 노드 생성
        print("3. 노드 생성 중...")
        create_departments_nodes(neo4j_manager, graph_data['departments'])
        create_employees_nodes(neo4j_manager, graph_data['employees'])
        create_projects_nodes(neo4j_manager, graph_data['projects'])
        
        # 4. 기본 관계 생성
        print("4. 기본 관계 생성 중...")
        create_employee_department_relationships(neo4j_manager)
        
        # 5. 다른 관계들 생성 (함수가 정의되어 있다면)
        if 'create_hierarchy_relationships' in globals() and graph_data['hierarchy_relationships']:
            print("5. 계층 관계 생성 중...")
            create_hierarchy_relationships(neo4j_manager, graph_data['hierarchy_relationships'])
        
        if 'create_collaboration_relationships' in globals() and graph_data['collaboration_relationships']:
            print("6. 협업 관계 생성 중...")
            create_collaboration_relationships(neo4j_manager, graph_data['collaboration_relationships'])
        
        if 'create_project_participation_relationships' in globals() and graph_data['project_relationships']:
            print("7. 프로젝트 참여 관계 생성 중...")
            create_project_participation_relationships(neo4j_manager, graph_data['project_relationships'])
        
        # 통계 확인
        print("8. 그래프 통계 확인 중...")
        get_basic_statistics(neo4j_manager)
        
        print("\n✅ Neo4j 그래프 생성 완료!")
        return neo4j_manager
        
    except Exception as e:
        print(f"\n❌ 그래프 생성 실패: {str(e)}")
        
        # 상세 오류 정보
        import traceback
        print("\n상세 오류 정보:")
        print(traceback.format_exc())
        
        # 연결 정리
        if neo4j_manager:
            try:
                neo4j_manager.close()
            except:
                pass
            neo4j_manager = None
        
        return None

def get_basic_statistics(neo4j_manager):
    """기본 통계 확인"""
    try:
        with neo4j_manager.driver.session() as session:
            # 노드 수 확인
            result = session.run("MATCH (n) RETURN labels(n)[0] as type, count(n) as count")
            print("\n생성된 노드:")
            for record in result:
                node_type = record['type'] if record['type'] else 'Unknown'
                print(f"  {node_type}: {record['count']}개")
            
            # 관계 수 확인
            result = session.run("MATCH ()-[r]->() RETURN type(r) as rel_type, count(r) as count")
            print("\n생성된 관계:")
            for record in result:
                print(f"  {record['rel_type']}: {record['count']}개")
                
    except Exception as e:
        print(f"통계 조회 실패: {str(e)}")

# 필수 함수들 정의 (셀 3이 실행되지 않았을 경우를 대비)
def create_employees_nodes(neo4j_manager, employees_data):
    """직원 노드 생성"""
    query = """
    UNWIND $employees as employee
    CREATE (e:Employee {
        employee_id: employee.employee_id,
        name: employee.name,
        department: employee.department,
        job_role: employee.job_role,
        job_level: employee.job_level,
        years_at_company: employee.years_at_company,
        risk_tier: employee.risk_tier,
        persona: employee.persona,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, employees=employees_data)
        logger.info(f"직원 노드 {len(employees_data)}개 생성 완료")

def create_departments_nodes(neo4j_manager, departments_data):
    """부서 노드 생성"""
    query = """
    UNWIND $departments as dept
    CREATE (d:Department {
        name: dept.name,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, departments=departments_data)
        logger.info(f"부서 노드 {len(departments_data)}개 생성 완료")

def create_projects_nodes(neo4j_manager, projects_data):
    """프로젝트 노드 생성"""
    query = """
    UNWIND $projects as project
    CREATE (p:Project {
        project_id: project.project_id,
        project_name: project.project_name,
        project_type: project.project_type,
        team_size: project.team_size,
        start_date: date(project.start_date),
        end_date: date(project.end_date),
        status: project.status,
        priority: project.priority,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, projects=projects_data)
        logger.info(f"프로젝트 노드 {len(projects_data)}개 생성 완료")

def create_employee_department_relationships(neo4j_manager):
    """직원-부서 관계 생성"""
    query = """
    MATCH (e:Employee), (d:Department)
    WHERE e.department = d.name
    CREATE (e)-[:WORKS_IN {
        created_at: datetime()
    }]->(d)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        summary = result.consume()
        logger.info(f"직원-부서 관계 {summary.counters.relationships_created}개 생성 완료")

print("=== 수정된 Neo4j 그래프 생성 실행 ===")

# 데이터 상태 확인
print("데이터 확인:")
print(f"  직원 데이터: {len(graph_data['employees'])}명")
print(f"  부서 데이터: {len(graph_data['departments'])}개")
print(f"  프로젝트 데이터: {len(graph_data['projects'])}개")

# 그래프 생성 실행
neo4j_manager = create_neo4j_graph_fixed(graph_data, NEO4J_CONFIG)

if neo4j_manager:
    print(f"\n🎉 성공! neo4j_manager가 생성되었습니다.")
    print(f"   연결 상태: 활성")
    print(f"   서버: {NEO4J_CONFIG['uri']}")
else:
    print(f"\n⚠️  neo4j_manager 생성에 실패했습니다.")
    print("   Neo4j 서버 상태와 연결 정보를 확인하세요.")

노드 생성 함수들 정의 완료


In [ ]:
# ------------------------------------------------------
# Neo4j 관계 생성 함수들 (수정됨 - 배치 처리 및 연결 복구 추가)
# ------------------------------------------------------

import time
from neo4j.exceptions import ServiceUnavailable, SessionExpired

def create_hierarchy_relationships(neo4j_manager, hierarchy_data):
    """계층 관계 생성 (상사-부하)"""
    
    query = """
    UNWIND $relationships as rel
    MATCH (sub:Employee {employee_id: rel.subordinate_id})
    MATCH (mgr:Employee {employee_id: rel.manager_id})
    CREATE (sub)-[:REPORTS_TO {
        reporting_frequency: rel.reporting_frequency,
        relationship_start: date(rel.relationship_start),
        created_at: datetime()
    }]->(mgr)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, relationships=hierarchy_data)
        summary = result.consume()
        logger.info(f"계층 관계 {summary.counters.relationships_created}개 생성 완료")

def create_collaboration_relationships_batch(neo4j_manager, collaboration_data, batch_size=1000):
    """협업 관계 생성 (배치 처리 및 연결 복구 기능 추가)"""
    
    # 중복 제거: 양방향 관계를 단방향으로 변환
    print("협업 관계 중복 제거 중...")
    unique_collaborations = []
    seen_pairs = set()
    
    for collab in collaboration_data:
        emp1, emp2 = collab['employee1_id'], collab['employee2_id']
        pair = tuple(sorted([emp1, emp2]))
        
        if pair not in seen_pairs:
            seen_pairs.add(pair)
            # 항상 작은 ID에서 큰 ID로 방향 설정 (일관성 유지)
            if emp1 < emp2:
                unique_collaborations.append(collab)
            else:
                # ID 순서를 바꿔서 저장
                collab_reversed = collab.copy()
                collab_reversed['employee1_id'] = emp2
                collab_reversed['employee2_id'] = emp1
                unique_collaborations.append(collab_reversed)
    
    print(f"중복 제거 완료: {len(collaboration_data):,} → {len(unique_collaborations):,} 관계")
    
    # 배치 처리 쿼리
    query = """
    UNWIND $relationships as rel
    MATCH (e1:Employee {employee_id: rel.employee1_id})
    MATCH (e2:Employee {employee_id: rel.employee2_id})
    CREATE (e1)-[:COLLABORATES_WITH {
        collaboration_strength: rel.collaboration_strength,
        interaction_frequency: rel.interaction_frequency,
        collaboration_type: rel.collaboration_type,
        relationship_quality: rel.relationship_quality,
        common_projects: rel.common_projects,
        is_bidirectional: true,
        created_at: datetime()
    }]->(e2)
    """
    
    total_created = 0
    total_batches = (len(unique_collaborations) + batch_size - 1) // batch_size
    
    for i in range(0, len(unique_collaborations), batch_size):
        batch = unique_collaborations[i:i + batch_size]
        batch_num = (i // batch_size) + 1
        
        retry_count = 0
        max_retries = 3
        
        while retry_count < max_retries:
            try:
                with neo4j_manager.driver.session() as session:
                    result = session.run(query, relationships=batch)
                    summary = result.consume()
                    created_count = summary.counters.relationships_created
                    total_created += created_count
                    
                    print(f"배치 {batch_num}/{total_batches} 완료: {created_count:,}개 관계 생성 (누적: {total_created:,})")
                    break
                    
            except (ServiceUnavailable, SessionExpired) as e:
                retry_count += 1
                wait_time = 2 ** retry_count  # 지수 백오프
                logger.warning(f"배치 {batch_num} 연결 오류 (시도 {retry_count}/{max_retries}): {str(e)}")
                
                if retry_count < max_retries:
                    print(f"  → {wait_time}초 후 재시도...")
                    time.sleep(wait_time)
                    
                    # 새로운 연결 시도
                    try:
                        neo4j_manager.driver.verify_connectivity()
                    except:
                        print("  → 연결 복구 중...")
                        time.sleep(5)
                else:
                    logger.error(f"배치 {batch_num} 최대 재시도 횟수 초과")
                    raise
            
            except Exception as e:
                logger.error(f"배치 {batch_num} 처리 중 예상치 못한 오류: {str(e)}")
                raise
        
        # 배치 간 잠시 대기 (서버 부하 방지)
        if batch_num % 10 == 0:  # 10배치마다 잠시 대기
            time.sleep(1)
    
    logger.info(f"협업 관계 생성 완료: 총 {total_created:,}개")
    return total_created

def create_project_participation_relationships(neo4j_manager, project_relationships_data):
    """프로젝트 참여 관계 생성"""
    
    query = """
    UNWIND $relationships as rel
    MATCH (e:Employee {employee_id: rel.employee_id})
    MATCH (p:Project {project_id: rel.project_id})
    CREATE (e)-[:PARTICIPATES_IN {
        role_in_project: rel.role_in_project,
        contribution_level: rel.contribution_level,
        project_start: date(rel.project_start),
        project_end: date(rel.project_end),
        project_status: rel.project_status,
        created_at: datetime()
    }]->(p)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, relationships=project_relationships_data)
        summary = result.consume()
        logger.info(f"프로젝트 참여 관계 {summary.counters.relationships_created}개 생성 완료")

def create_project_collaboration_relationships(neo4j_manager):
    """프로젝트 내 팀원 간 협업 관계 생성 (수정: 방향성 있는 관계)"""
    
    query = """
    MATCH (e1:Employee)-[:PARTICIPATES_IN]->(p:Project)<-[:PARTICIPATES_IN]-(e2:Employee)
    WHERE e1.employee_id < e2.employee_id
    CREATE (e1)-[:PROJECT_COLLEAGUE {
        shared_project: p.project_id,
        project_name: p.project_name,
        is_bidirectional: true,
        created_at: datetime()
    }]->(e2)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        summary = result.consume()
        logger.info(f"프로젝트 내 협업 관계 {summary.counters.relationships_created}개 생성 완료")

def create_team_relationships(neo4j_manager):
    """같은 팀(부서+직급) 관계 생성"""
    
    query = """
    MATCH (e1:Employee), (e2:Employee)
    WHERE e1.department = e2.department 
    AND e1.job_level = e2.job_level 
    AND e1.employee_id < e2.employee_id
    CREATE (e1)-[:SAME_TEAM {
        department: e1.department,
        job_level: e1.job_level,
        is_bidirectional: true,
        created_at: datetime()
    }]->(e2)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        summary = result.consume()
        logger.info(f"같은 팀 관계 {summary.counters.relationships_created}개 생성 완료")

print("관계 생성 함수들 정의 완료 (수정됨)")

관계 생성 함수들 정의 완료 (수정됨)


In [ ]:
# ------------------------------------------------------
# Neo4j 데이터베이스 생성 실행 (수정됨)
# ------------------------------------------------------

def create_neo4j_graph(graph_data, neo4j_config):
    """전체 Neo4j 그래프 생성 프로세스 (수정됨)"""
    
    neo4j_manager = None
    
    try:
        # Neo4j 연결
        neo4j_manager = Neo4jManager(
            neo4j_config['uri'], 
            neo4j_config['username'], 
            neo4j_config['password']
        )
        
        # 1. 데이터베이스 초기화
        print("1. 기존 데이터 삭제 중...")
        neo4j_manager.clear_database()
        
        # 2. 제약조건 및 인덱스 생성
        print("2. 제약조건 및 인덱스 생성 중...")
        neo4j_manager.create_constraints_and_indexes()
        
        # 3. 노드 생성
        print("3. 노드 생성 중...")
        create_departments_nodes(neo4j_manager, graph_data['departments'])
        create_employees_nodes(neo4j_manager, graph_data['employees'])
        create_projects_nodes(neo4j_manager, graph_data['projects'])
        
        # 4. 기본 관계 생성 (직원-부서)
        print("4. 기본 관계 생성 중...")
        create_employee_department_relationships(neo4j_manager)
        
        # 5. 조직 관계 생성
        print("5. 조직 관계 생성 중...")
        
        # 연결 상태 확인
        if not neo4j_manager.verify_connection():
            raise Exception("Neo4j 연결 복구 실패")
        
        if graph_data['hierarchy_relationships']:
            create_hierarchy_relationships(neo4j_manager, graph_data['hierarchy_relationships'])
        
        if graph_data['collaboration_relationships']:
            print(f"협업 관계 처리 시작: {len(graph_data['collaboration_relationships']):,}개")
            create_collaboration_relationships_batch(neo4j_manager, graph_data['collaboration_relationships'], batch_size=500)
        
        if graph_data['project_relationships']:
            create_project_participation_relationships(neo4j_manager, graph_data['project_relationships'])
        
        # 6. 파생 관계 생성
        print("6. 파생 관계 생성 중...")
        create_project_collaboration_relationships(neo4j_manager)
        create_team_relationships(neo4j_manager)  # 새로 추가
        
        # 7. 생성된 그래프 통계
        print("7. 그래프 통계 확인 중...")
        get_graph_statistics(neo4j_manager)
        
        print("\n✅ Neo4j 그래프 생성 완료!")
        
        return neo4j_manager
        
    except Exception as e:
        logger.error(f"Neo4j 그래프 생성 실패: {str(e)}")
        if neo4j_manager:
            neo4j_manager.close()
        raise

def get_graph_statistics(neo4j_manager):
    """생성된 그래프의 통계 정보 조회 (수정됨)"""
    
    statistics_queries = {
        "총 노드 수": "MATCH (n) RETURN count(n) as count",
        "총 관계 수": "MATCH ()-[r]->() RETURN count(r) as count",
        "직원 노드 수": "MATCH (e:Employee) RETURN count(e) as count",
        "부서 노드 수": "MATCH (d:Department) RETURN count(d) as count", 
        "프로젝트 노드 수": "MATCH (p:Project) RETURN count(p) as count",
        "계층 관계 수": "MATCH ()-[r:REPORTS_TO]->() RETURN count(r) as count",
        "협업 관계 수 (단방향)": "MATCH ()-[r:COLLABORATES_WITH]->() RETURN count(r) as count",
        "협업 관계 수 (양방향 계산)": "MATCH ()-[r:COLLABORATES_WITH]->() RETURN count(r) * 2 as count",
        "프로젝트 참여 관계 수": "MATCH ()-[r:PARTICIPATES_IN]->() RETURN count(r) as count",
        "프로젝트 내 협업 관계 수": "MATCH ()-[r:PROJECT_COLLEAGUE]->() RETURN count(r) as count",
        "같은 팀 관계 수": "MATCH ()-[r:SAME_TEAM]->() RETURN count(r) as count"
    }
    
    print("\n=== Neo4j 그래프 통계 ===")
    
    with neo4j_manager.driver.session() as session:
        for description, query in statistics_queries.items():
            try:
                result = session.run(query)
                count = result.single()["count"]
                print(f"  {description}: {count:,}")
            except Exception as e:
                print(f"  {description}: 조회 실패 - {str(e)}")

# Neo4j 그래프 생성 실행
try:
    print("Neo4j 그래프 생성 시작...")
    print(f"Neo4j 서버: {NEO4J_CONFIG['uri']}")
    
    # 실제 실행
    neo4j_manager = create_neo4j_graph(graph_data, NEO4J_CONFIG)
    
except Exception as e:
    print(f"\n❌ Neo4j 연결 실패: {str(e)}")
    print("\n해결 방법:")
    print("1. Neo4j Desktop 또는 Community Server가 실행 중인지 확인")
    print("2. 연결 정보(URI, username, password)가 올바른지 확인")
    print("3. 방화벽에서 7687 포트가 열려있는지 확인")
    print("\n설정을 확인한 후 다시 실행하세요.")
    neo4j_manager = None

INFO:__main__:Neo4j 연결 성공


Neo4j 그래프 생성 시작...
Neo4j 서버: bolt://18.208.202.177:7687


INFO:__main__:Neo4j 서버 응답 확인


1. 기존 데이터 삭제 중...


INFO:__main__:데이터베이스 초기화 완료
INFO:__main__:실행 완료: CREATE CONSTRAINT IF NOT EXISTS FOR (e:Employee) REQUIRE e.employee_id IS UNIQUE


...

4. 기본 관계 생성 중...


INFO:__main__:직원-부서 관계 1470개 생성 완료


5. 조직 관계 생성 중...


INFO:__main__:계층 관계 295개 생성 완료
ERROR:neo4j:Failed to read from defunct connection IPv4Address(('18.208.202.177', 7687)) (IPv4Address(('18.208.202.177', 7687)))
ERROR:__main__:Neo4j 그래프 생성 실패: Failed to read from defunct connection IPv4Address(('18.208.202.177', 7687)) (IPv4Address(('18.208.202.177', 7687)))
INFO:__main__:Neo4j 연결 종료



❌ Neo4j 연결 실패: Failed to read from defunct connection IPv4Address(('18.208.202.177', 7687)) (IPv4Address(('18.208.202.177', 7687)))

해결 방법:
1. Neo4j Desktop 또는 Community Server가 실행 중인지 확인
2. 연결 정보(URI, username, password)가 올바른지 확인
3. 방화벽에서 7687 포트가 열려있는지 확인

설정을 확인한 후 다시 실행하세요.


In [7]:
# ------------------------------------------------------
# Neo4j Browser 시각화 가이드
# ------------------------------------------------------

def generate_visualization_queries():
    """Neo4j Browser에서 사용할 시각화 쿼리들"""
    
    viz_queries = {
        "전체 조직 구조 (샘플)": """
            // 한 부서의 계층 구조 시각화
            MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: 'Sales'})
            OPTIONAL MATCH (e)-[r:REPORTS_TO]->(manager)
            RETURN e, r, manager, d
            LIMIT 20
        """,
        
        "고위험 직원과 그 관계": """
            // 고위험 직원과 연결된 모든 관계
            MATCH (high_risk:Employee {risk_tier: 'High'})
            OPTIONAL MATCH (high_risk)-[r1]-(connected)
            RETURN high_risk, r1, connected
            LIMIT 50
        """,
        
        "프로젝트 협업 네트워크": """
            // 특정 프로젝트의 팀원과 그들의 협업 관계
            MATCH (p:Project {status: 'active'})<-[:PARTICIPATES_IN]-(e:Employee)
            OPTIONAL MATCH (e)-[c:COLLABORATES_WITH]-(colleague)-[:PARTICIPATES_IN]->(p)
            RETURN p, e, c, colleague
            LIMIT 30
        """,
        
        "부서간 협업 네트워크": """
            // 서로 다른 부서 직원들 간의 협업 관계
            MATCH (e1:Employee)-[:WORKS_IN]->(d1:Department)
            MATCH (e2:Employee)-[:WORKS_IN]->(d2:Department)
            MATCH (e1)-[c:COLLABORATES_WITH]-(e2)
            WHERE d1 <> d2 AND c.collaboration_strength > 0.6
            RETURN e1, e2, c, d1, d2
            LIMIT 25
        """,
        
        "위험 전파 경로 분석": """
            // 고위험 직원으로부터 2단계 이내의 관계
            MATCH (high_risk:Employee {risk_tier: 'High'})
            MATCH path = (high_risk)-[*1..2]-(connected:Employee)
            WHERE connected.risk_tier IN ['High', 'Neutral']
            RETURN path
            LIMIT 40
        """
    }
    
    print("=== Neo4j Browser 시각화 가이드 ===\n")
    
    print("1. Neo4j Browser 접속:")
    print("   http://localhost:7474 (기본 주소)")
    print("   username: neo4j")
    print("   password: [설정한 비밀번호]\n")
    
    print("2. 아래 쿼리들을 복사해서 Neo4j Browser에서 실행하세요:\n")
    
    for name, query in viz_queries.items():
        print(f"🎨 {name}:")
        print("```cypher")
        print(query.strip())
        print("```\n")
    
    print("3. 시각화 팁:")
    print("   - 노드 크기: 직급(job_level) 또는 위험도(risk_tier)에 따라 조정")
    print("   - 노드 색상: 부서별 또는 위험등급별로 구분")
    print("   - 관계 두께: 협업 강도(collaboration_strength)에 따라 조정")
    print("   - 레이아웃: Force-directed 또는 Hierarchical 레이아웃 사용")
    
    return viz_queries

# 시각화 쿼리 생성
viz_queries = generate_visualization_queries()

# Neo4j 연결 정리
if 'neo4j_manager' in locals() and neo4j_manager:
    print(f"\n=== Neo4j 그래프 생성 완료 ===")
    print(f"✅ 총 {len(graph_data['employees'])}명의 직원 데이터가 Neo4j에 저장되었습니다.")
    print(f"✅ 계층관계, 협업관계, 프로젝트 참여 관계가 모두 생성되었습니다.")
    print(f"\n🌐 Neo4j Browser에서 그래프를 시각적으로 탐색해보세요:")
    print(f"   http://localhost:7474")
    
    # 필요한 경우 연결 종료
    # neo4j_manager.close()
else:
    print("\n⚠️  Neo4j 연결 설정을 완료한 후 다시 실행하세요.")
    print("   1. Neo4j 서버 실행")
    print("   2. 인증 정보 수정")
    print("   3. 코드 재실행")

=== Neo4j Browser 시각화 가이드 ===

1. Neo4j Browser 접속:
   http://localhost:7474 (기본 주소)
   username: neo4j
   password: [설정한 비밀번호]

2. 아래 쿼리들을 복사해서 Neo4j Browser에서 실행하세요:

🎨 전체 조직 구조 (샘플):
```cypher
// 한 부서의 계층 구조 시각화
            MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: 'Sales'})
            OPTIONAL MATCH (e)-[r:REPORTS_TO]->(manager)
            RETURN e, r, manager, d
            LIMIT 20
```

🎨 고위험 직원과 그 관계:
```cypher
// 고위험 직원과 연결된 모든 관계
            MATCH (high_risk:Employee {risk_tier: 'High'})
            OPTIONAL MATCH (high_risk)-[r1]-(connected)
            RETURN high_risk, r1, connected
            LIMIT 50
```

🎨 프로젝트 협업 네트워크:
```cypher
// 특정 프로젝트의 팀원과 그들의 협업 관계
            MATCH (p:Project {status: 'active'})<-[:PARTICIPATES_IN]-(e:Employee)
            OPTIONAL MATCH (e)-[c:COLLABORATES_WITH]-(colleague)-[:PARTICIPATES_IN]->(p)
            RETURN p, e, c, colleague
            LIMIT 30
```

🎨 부서간 협업 네트워크:
```cypher
// 서로 다른 부서 직원들 간의 협업 관계
            MATCH (e1: